## Import

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
!pip install -q -U segmentation-models-pytorch albumentations > /dev/null

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/sw_contest"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/sw_contest


In [ ]:
!pip install --upgrade opencv-python

## Utils

In [ ]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## Custom Dataset

In [ ]:
class SatelliteDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_rle = self.data.iloc[idx, 2]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:

            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

## Data Loader

crop

In [ ]:
train_df = pd.read_csv("./train.csv")


In [ ]:
def crop_img(img, img_size=256):
    img_list = []

    y_cnt = 0
    while True:
        start_y = y_cnt * img_size
        end_y = (y_cnt + 1) * img_size

        if end_y > 1024:
            break

        x_cnt = 0
        while True:
            start_x = x_cnt * img_size
            end_x = (x_cnt + 1) * img_size

            if end_x > 1024:
                break

            temp_img = img[start_x:end_x, start_y:end_y, :]
            x_cnt += 1
            img_list.append(temp_img)

        y_cnt += 1

    return img_list

In [ ]:
# new_data_list = []
# for idx, row in train_df.iterrows():
#     img_name = row["img_path"].split("/")[-1]
#     img_path = os.path.join(".", img_name)

#     img = cv2.imread(img_path)

#     images = crop_img(img)
#     masks = crop_img(mask)

#     for idx, (img, mask) in enumerate(zip(images, masks)):
#         new_img_name = img_name[:-4] + "_" + str(idx).zfill(2) + ".png"
#         new_data_list.append({"img_id": new_img_name[:-4]})

#         cv2.imwrite(os.path.join("./train_img_crop", new_img_name), img)


TypeError: ignored

normal augmentation

In [ ]:
def train_transform(train = True):
        return A.Compose(
            [
                A.OneOf(
                    [
                        A.RandomBrightness(p=1),
                        A.RandomBrightnessContrast(p=1),
                        A.Emboss(p=1),
                        A.RandomShadow(p=1),
                        A.NoOp(),
                    ],
                    p=1,
                ),
                A.OneOf(
                    [
                        A.Blur(p=1),
                        A.AdvancedBlur(p=1),
                        A.MotionBlur(p=1),
                    ],
                    p=0.6,
                ),
                A.OneOf(
                    [
                        A.NoOp(),
                        A.HorizontalFlip(p=0.5),
                        A.VerticalFlip(p=0.5),
                        A.ShiftScaleRotate(p=0.5),
                        A.Rotate(limit=90, p=1, border_mode=cv2.BORDER_REPLICATE),
                        A.RandomRotate90(p=1)
                    ],
                    p=1,
                ),
                A.RandomCrop(224, 224),
                A.Normalize(),
                ToTensorV2(transpose_mask=True)
            ]
        )


def test_transform():
        return A.Compose(
            [
                A.Normalize(),
                ToTensorV2(transpose_mask=True)
            ]
        )

train_transform = train_transform()
test_transform = test_transform()

dataset = SatelliteDataset(csv_file='./train.csv', transform=train_transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1258: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


## Define Model

## Model Train

DeepLabV3Plus

In [ ]:
# model 초기화
import segmentation_models_pytorch as smp

lr = 1e-4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 1

num_classes = 2

# 전이 학습에 사용할 미리 학습된 DeepLabV3Plus 모델 가져오기
model = smp.DeepLabV3Plus(encoder_name="resnet101", encoder_weights="imagenet", in_channels=3, classes=num_classes)


# 모델을 GPU로 이동
model.to(device)


v3plus_weight = torch.load("./weights/v3plus_11.pth")
model.load_state_dict(v3plus_weight, strict=False)
# v3plus_weight = torch.load("./songweights/v3plus3_30.pth")
# model.load_state_dict(v3plus_weight,strict=False)

# loss function과 optimizer 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr)

# training loop
for epoch in range(num_epochs):  # 30 에폭 동안 학습합니다.
    model.train()
    epoch_loss = 0
    for images, masks in tqdm(dataloader):
        images = images.to(device)
        masks = masks.to(device)
        masks = masks.long()

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

   # if (epoch+1)%10 == 0:
    torch.save(model.state_dict(), './weights/v3plus_' + str(epoch+1+11) + '.pth')


    print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader)}')


  2%|▏         | 4/224 [00:52<47:43, 13.02s/it]


KeyboardInterrupt: ignored

UnetPlusPlus

In [ ]:
# model 초기화
import segmentation_models_pytorch as smp

lr = 1e-4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 21

num_classes = 2

# 전이 학습에 사용할 미리 학습된 DeepLabV3Plus 모델 가져오기
model2 = smp.UnetPlusPlus(encoder_name="resnet101", encoder_weights="imagenet", in_channels=3, classes=num_classes)


# 모델을 GPU로 이동
model2.to(device)

# v3plus_weight = torch.load("./songweights/v3plus3_30.pth")
# model.load_state_dict(v3plus_weight,strict=False)

# loss function과 optimizer 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr)

# training loop
for epoch in range(num_epochs):  # 30 에폭 동안 학습합니다.
    model2.train()
    epoch_loss = 0
    for images, masks in tqdm(dataloader):
        images = images.to(device)
        masks = masks.to(device)
        masks = masks.long()

        optimizer.zero_grad()
        outputs = model2(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    if (epoch+1)%10 == 1:
      torch.save(model2.state_dict(), './JOngweights/unetplus_' + str(epoch+1) + '.pth')


    print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader)}')


  1%|▏         | 3/224 [00:51<1:03:14, 17.17s/it]


KeyboardInterrupt: ignored

## Inference

In [ ]:
# with torch.no_grad():
#     model.eval()
#     result = []
#     for images in tqdm(test_dataloader):
#         images = images.float().to(device)

#         outputs = model(images)
#         masks = torch.sigmoid(outputs).cpu().numpy()
#         masks = np.squeeze(masks, axis=1)
#         masks = (masks > 0.35).astype(np.uint8) # Threshold = 0.35

#         for i in range(len(images)):
#             mask_rle = rle_encode(masks[i])
#             if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
#                 result.append(-1)
#             else:
#                 result.append(mask_rle)

In [ ]:
v3plus_weight = torch.load("./weights/v3plus_21.pth")
model.load_state_dict(v3plus_weight, strict=False)

unetplus_weight = torch.load("./weights/unetplus_21_song.pth")
model2.load_state_dict(unetplus_weight, strict=False)

test_dataset = SatelliteDataset(csv_file='./test.csv', transform=test_transform, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

with torch.no_grad():
    model.eval()
    model2.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(device)

        outputs1 = model(images)
        masks1 = torch.sigmoid(outputs1).cpu().numpy()

        outputs2 = model2(images)
        masks2 = torch.sigmoid(outputs2).cpu().numpy()

        # 예측 결과를 보팅 결과에 추가
        voting_results = []
        for i in range(len(images)):
            mask1 = (masks1[i] > 0.35).astype(np.uint8)  # 임계값 = 0.35
            mask2 = (masks2[i] > 0.35).astype(np.uint8)  # 임계값 = 0.35

            # 두 모델의 예측을 모두 사용하여 보팅을 수행
            combined_votes = mask1 + mask2
            final_mask = (combined_votes >= 2).astype(np.uint8)  # 두 모델이 동일하게 예측한 부분만 선택
            mask_rle = rle_encode(final_mask)
            result.append(mask_rle)

submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result
submit.to_csv("submission_ensemble.csv", index=False)


100%|██████████| 1895/1895 [2:36:04<00:00,  4.94s/it]


## Submission

In [ ]:
submit = pd.read_csv('./sample_submission_song.csv')
submit['mask_rle'] = result

In [ ]:
submit.to_csv('./submit_deeplabveplus_resnet101_epoch30.csv', index=False)

In [ ]:
import pandas as pd

# CSV 파일을 읽어와 DataFrame에 저장
csv_file_path = './submission_ensemble.csv'  # CSV 파일의 경로를 지정해야 합니다.
df = pd.read_csv(csv_file_path)

# 'mask_rle' 열에서 -1인 행의 총 수를 출력
count_minus_1 = df['mask_rle'].eq('1 50176').sum()

print("Total number of rows with -1 in 'mask_rle' column:", count_minus_1)


Total number of rows with -1 in 'mask_rle' column: 15429
